In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from tqdm import tqdm
import numpy as np
import os
import xml.etree.ElementTree as ET
from PIL import Image
import os
import torch
import torchvision
import torchvision.transforms as T
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np

In [3]:
# Paths for annotations and images
annotations_path = '/content/drive/MyDrive/mini project/annotations'
images_path = '/content/drive/MyDrive/mini project/images'



In [4]:
# import torch
# import torchvision
# import torchvision.transforms as T
# from torchvision.models.detection import FasterRCNN
# from torchvision.models.detection.rpn import AnchorGenerator
# from torch.utils.data import Dataset, DataLoader
# from tqdm import tqdm
# import xml.etree.ElementTree as ET
# from PIL import Image
# import numpy as np
# import os

# # Paths for annotations and images
# annotations_path = '/content/drive/MyDrive/mini project/annotations'
# images_path = '/content/drive/MyDrive/mini project/images'

# # ==========================
# # 1. Dataset Preparation
# # ==========================
# class UATDDataset(Dataset):
#     def __init__(self, image_dir, annotation_dir, transforms=None):
#         self.image_dir = image_dir
#         self.annotation_dir = annotation_dir
#         self.transforms = transforms
#         self.images = sorted([f for f in os.listdir(self.image_dir) if f.endswith('.bmp')])
#         self.annotations = sorted([f for f in os.listdir(self.annotation_dir) if f.endswith('.xml')])

#         assert len(self.images) == len(self.annotations), "Mismatch between images and annotations count!"

#     def __getitem__(self, idx):
#         # Paths to image and annotation
#         img_path = os.path.join(self.image_dir, self.images[idx])
#         ann_path = os.path.join(self.annotation_dir, self.annotations[idx])

#         # Load image
#         img = Image.open(img_path).convert("RGB")

#         # Parse annotation
#         target = self.parse_annotations(ann_path)

#         # Apply transforms
#         if self.transforms:
#             img = self.transforms(img)

#         return img, target

#     def __len__(self):
#         return len(self.images)

#     def parse_annotations(self, path):
#         """Parses XML annotations into a target dictionary."""
#         tree = ET.parse(path)
#         root = tree.getroot()
#         boxes = []
#         labels = []

#         # Assuming each object is labeled with a unique class
#         class_map = {
#             'class1': 1, 'class2': 2, 'class3': 3, 'class4': 4, 'class5': 5,
#             'class6': 6, 'class7': 7, 'class8': 8, 'class9': 9, 'class10': 10
#         }

#         for obj in root.findall('object'):
#             class_name = obj.find('name').text
#             label = class_map.get(class_name, 0)  # Default to 0 if the class name doesn't match
#             if label > 0:
#                 labels.append(label)
#                 bndbox = obj.find('bndbox')
#                 xmin = int(bndbox.find('xmin').text)
#                 ymin = int(bndbox.find('ymin').text)
#                 xmax = int(bndbox.find('xmax').text)
#                 ymax = int(bndbox.find('ymax').text)
#                 boxes.append([xmin, ymin, xmax, ymax])

#         target = {
#             'boxes': torch.tensor(boxes, dtype=torch.float32),
#             'labels': torch.tensor(labels, dtype=torch.int64)
#         }
#         return target



In [5]:

class UATDDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transforms=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transforms = transforms
        self.images = sorted([f for f in os.listdir(self.image_dir) if f.endswith('.bmp')])
        self.annotations = sorted([f for f in os.listdir(self.annotation_dir) if f.endswith('.xml')])

        assert len(self.images) == len(self.annotations), "Mismatch between images and annotations count!"

    def __getitem__(self, idx):
        # Paths to image and annotation
        img_path = os.path.join(self.image_dir, self.images[idx])
        ann_path = os.path.join(self.annotation_dir, self.annotations[idx])

        # Load image
        img = Image.open(img_path).convert("RGB")

        # Parse annotation
        target = self.parse_annotations(ann_path)

        # If no objects are found, return empty tensors
        if target['boxes'].shape[0] == 0:
            target['boxes'] = torch.empty((0, 4))
            target['labels'] = torch.empty(0, dtype=torch.int64)

        # Apply transforms
        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.images)
    def parse_annotations(self, path):
            """Parses XML annotations into a target dictionary."""
            tree = ET.parse(path)
            root = tree.getroot()
            boxes = []
            labels = []

            for obj in root.findall('object'):
                label = 1  # Assuming a single class; adjust for multi-class datasets
                labels.append(label)
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                boxes.append([xmin, ymin, xmax, ymax])

            print(f"Boxes: {boxes}, Labels: {labels}")  # Debug print statement

            if len(boxes) == 0:
                print(f"No objects found in annotation file: {path}")  # Debug print for empty annotations

            target = {
                'boxes': torch.tensor(boxes, dtype=torch.float32),
                'labels': torch.tensor(labels, dtype=torch.int64)
            }
            return target


In [6]:
# def parse_annotations(self, path):
#     """Parses XML annotations into a target dictionary."""
#     tree = ET.parse(path)
#     root = tree.getroot()
#     boxes = []
#     labels = []

#     for obj in root.findall('object'):
#         label = 1  # Assuming a single class; adjust for multi-class datasets
#         labels.append(label)
#         bndbox = obj.find('bndbox')
#         xmin = int(bndbox.find('xmin').text)
#         ymin = int(bndbox.find('ymin').text)
#         xmax = int(bndbox.find('xmax').text)
#         ymax = int(bndbox.find('ymax').text)
#         boxes.append([xmin, ymin, xmax, ymax])

#     print(f"Boxes: {boxes}, Labels: {labels}")  # Debug print statement

#     if len(boxes) == 0:
#         print(f"No objects found in annotation file: {path}")  # Debug print for empty annotations

#     target = {
#         'boxes': torch.tensor(boxes, dtype=torch.float32),
#         'labels': torch.tensor(labels, dtype=torch.int64)
#     }
#     return target


In [7]:
def get_transforms():
    """Return transformations for the dataset."""
    return T.Compose([
        T.ToTensor(),
        T.Resize((512, 512)),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])



In [8]:

# ==========================
# 2. Model Setup
# ==========================
def get_faster_rcnn_model(backbone='resnet50', num_classes=11):  # 10 classes + background
    """Return a Faster-RCNN model with the specified backbone."""
    if backbone == 'resnet18':
        backbone_model = torchvision.models.resnet18(pretrained=True)
    elif backbone == 'resnet50':
        backbone_model = torchvision.models.resnet50(pretrained=True)
    elif backbone == 'resnet101':
        backbone_model = torchvision.models.resnet101(pretrained=True)
    else:
        raise ValueError("Invalid backbone choice!")

    # Use backbone without the classification head
    backbone_model = torch.nn.Sequential(*list(backbone_model.children())[:-2])
    backbone_model.out_channels = 2048

    # RPN and ROI pooling
    anchor_generator = AnchorGenerator(
        sizes=((32, 64, 128, 256, 512),),
        aspect_ratios=((0.5, 1.0, 2.0),)
    )
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'],
        output_size=7,
        sampling_ratio=2
    )

    # Define FasterRCNN model
    model = FasterRCNN(
        backbone_model,
        num_classes=num_classes,  # Number of classes (including background)
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler
    )
    return model



In [9]:

# ==========================
# 3. Training and Validation
# ==========================
def train_one_epoch(model, optimizer, data_loader, device):
    model.train()
    epoch_loss = 0
    for images, targets in tqdm(data_loader):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        epoch_loss += losses.item()

    return epoch_loss / len(data_loader)

def validate_one_epoch(model, data_loader, device):
    model.eval()
    metrics = []
    with torch.no_grad():
        for images, targets in tqdm(data_loader):
            images = [img.to(device) for img in images]
            outputs = model(images)
            # Compute validation metrics (Placeholder: Replace with actual evaluation code)
            metrics.append(0)  # Placeholder
    return np.mean(metrics)



In [10]:

# ==========================
# 4. Main Training Loop
# ==========================
def main():
    # Configurations
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch_size = 3
    num_epochs = 10
    backbone = 'resnet50'

    # Paths
    image_dir = '/content/drive/MyDrive/mini project/images'
    annotation_dir = '/content/drive/MyDrive/mini project/annotations'

    # Dataset and DataLoader
    dataset_train = UATDDataset(image_dir, annotation_dir, transforms=get_transforms())
    train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

    # Model, Optimizer
    model = get_faster_rcnn_model(backbone, num_classes=11).to(device)  # 10 classes + background
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training Loop
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train_loss = train_one_epoch(model, optimizer, train_loader, device)
        print(f"Train Loss: {train_loss:.4f}")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 102MB/s]


Epoch 1/10


  0%|          | 0/17 [00:00<?, ?it/s]

Boxes: [[366, 1528, 420, 1599]], Labels: [1]
Boxes: [[424, 707, 478, 756], [376, 890, 448, 941]], Labels: [1, 1]
Boxes: [[224, 1244, 266, 1309]], Labels: [1]


  6%|▌         | 1/17 [00:09<02:24,  9.02s/it]

...

Boxes: [[392, 779, 540, 829]], Labels: [1]
Boxes: [[426, 1205, 492, 1269], [406, 1020, 466, 1082]], Labels: [1, 1]
Boxes: [[406, 1180, 452, 1246]], Labels: [1]


 94%|█████████▍| 16/17 [00:10<00:00,  1.59it/s]

Boxes: [[424, 707, 478, 756], [376, 890, 448, 941]], Labels: [1, 1]
Boxes: [[650, 777, 704, 817]], Labels: [1]


100%|██████████| 17/17 [00:10<00:00,  1.59it/s]

Train Loss: 7.8982


In [11]:

# ==========================
# 4. Main Training Loop
# ==========================
def main():
    # Configurations
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch_size = 8
    num_epochs = 10
    backbone = 'resnet50'

    # Paths
    image_dir = '/content/drive/MyDrive/mini project/images'
    annotation_dir = '/content/drive/MyDrive/mini project/annotations'

    # Dataset and DataLoader
    dataset_train = UATDDataset(image_dir, annotation_dir, transforms=get_transforms())
    train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

    # Model, Optimizer
    model = get_faster_rcnn_model(backbone, num_classes=11).to(device)  # 10 classes + background
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training Loop
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train_loss = train_one_epoch(model, optimizer, train_loader, device)
        print(f"Train Loss: {train_loss:.4f}")
    # Plot training loss over epochs
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), epoch_losses, label='Training Loss', color='blue')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.show()


if __name__ == "__main__":
    main()


Epoch 1/10


  0%|          | 0/7 [00:00<?, ?it/s]

Boxes: [[574, 880, 618, 909]], Labels: [1]
Boxes: [[240, 1394, 334, 1478]], Labels: [1]
Boxes: [[336, 834, 388, 878]], Labels: [1]
Boxes: [[392, 779, 540, 829]], Labels: [1]
Boxes: [[650, 777, 704, 817]], Labels: [1]
Boxes: [[630, 672, 692, 751], [544, 1010, 640, 1084]], Labels: [1, 1]
Boxes: [[426, 1205, 492, 1269], [406, 1020, 466, 1082]], Labels: [1, 1]
Boxes: [[366, 467, 480, 508]], Labels: [1]


 14%|█▍        | 1/7 [00:02<00:14,  2.36s/it]

Boxes: [[474, 857, 530, 905]], Labels: [1]
Boxes: [[548, 981, 680, 1054], [382, 1098, 434, 1149]], Labels: [1, 1]
Boxes: [[746, 533, 788, 569]], Labels: [1]
Boxes: [[326, 540, 370, 577]], Labels: [1]
Boxes: [[268, 785, 422, 844]], Labels: [1]
Boxes: [[460, 624, 502, 660]], Labels: [1]
Boxes: [[304, 740, 358, 785], [408, 795, 504, 852]], Labels: [1, 1]
Boxes: [[726, 1235, 828, 1363]], Labels: [1]


...

Boxes: [[424, 707, 478, 756], [376, 890, 448, 941]], Labels: [1, 1]
Boxes: [[286, 779, 390, 834], [272, 702, 330, 743], [398, 563, 446, 596]], Labels: [1, 1, 1]
Boxes: [[880, 472, 922, 509]], Labels: [1]
Boxes: [[474, 752, 516, 788]], Labels: [1]
Boxes: [[528, 1435, 584, 1540], [550, 1253, 596, 1333]], Labels: [1, 1]
Boxes: [[370, 1101, 422, 1161], [554, 983, 676, 1054]], Labels: [1, 1]
Boxes: [[514, 981, 610, 1054], [422, 962, 492, 1014], [418, 773, 474, 828]], Labels: [1, 1, 1]
Boxes: [[706, 670, 766, 755]], Labels: [1]


 57%|█████▋    | 4/7 [00:08<00:06,  2.02s/it]

Boxes: [[408, 780, 516, 840]], Labels: [1]
Boxes: [[366, 1528, 420, 1599]], Labels: [1]
Boxes: [[188, 1442, 296, 1540]], Labels: [1]
Boxes: [[224, 1244, 266, 1309]], Labels: [1]
Boxes: [[410, 764, 524, 821]], Labels: [1]
Boxes: [[344, 783, 442, 842], [448, 673, 506, 724], [496, 817, 592, 887]], Labels: [1, 1, 1]
Boxes: [[342, 751, 390, 787]], Labels: [1]
Boxes: [[264, 715, 322, 761]], Labels: [1]


 57%|█████▋    | 4/7 [00:09<00:06,  2.28s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.53 GiB. GPU 0 has a total capacity of 14.75 GiB of which 695.06 MiB is free. Process 4743 has 14.07 GiB memory in use. Of the allocated memory 11.66 GiB is allocated by PyTorch, and 2.27 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)